<a href="https://colab.research.google.com/github/PawanKrGunjan/Natural-Language-Processing/blob/main/Audio/Build_a_music_genre_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets[audio]
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00


# Load the dataset

In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

# Split into train & test

In [3]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.2)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 799
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 200
    })
})

In [4]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn

<bound method ClassLabel.int2str of ClassLabel(names=['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'], id=None)>

# Load the Automatic Classification Pipeline

In [5]:
from transformers import pipeline
model_id = "PawanKrGunjan/distilhubert-finetuned-gtzan"

classifier = pipeline(
    "audio-classification",
    model=model_id,
)

In [6]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/rock/rock.00012.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/rock/rock.00012.wav',
  'array': array([-0.006073  ,  0.0144043 ,  0.04669189, ..., -0.00701904,
         -0.01977539, -0.02853394]),
  'sampling_rate': 22050},
 'genre': 9}

In [7]:
gtzan["train"][0]['audio']['sampling_rate']

22050

In [8]:
from IPython.display import Audio
import librosa


# Load the audio file using librosa or any other audio processing library you prefer
audio_path=gtzan["train"][0]['audio']['path']
sample_rate =gtzan["train"][0]['audio']['sampling_rate']

audio_data, _ = librosa.load(audio_path, sr=sample_rate)

# Display the audio using IPython.display.Audio
display(Audio(data=audio_data, rate=sample_rate))

In [9]:
pred_genre = classifier(gtzan["train"][0]['audio']['path'])
pred_genre

[{'score': 0.361706018447876, 'label': 'reggae'},
 {'score': 0.1734716296195984, 'label': 'pop'},
 {'score': 0.15519817173480988, 'label': 'disco'},
 {'score': 0.1250380277633667, 'label': 'blues'},
 {'score': 0.09665042906999588, 'label': 'hiphop'}]

In [10]:
id2label_fn(gtzan["train"][0]["genre"])

actual_genre = gtzan["train"][0]["genre"]

print('Predicted Genre: ',pred_genre[0]['label'].split('_')[-1])
print('Actual Genre :',id2label_fn(gtzan["train"][0]["genre"]))

Predicted Genre:  reggae
Actual Genre : rock


# Load the transformer AutoFeatureExtractor

In [11]:
from transformers import AutoFeatureExtractor

model_id ='openai/whisper-tiny'
model_id ='PawanKrGunjan/whisper-tiny-finetuned-gtzan'
model_id = "ntu-spml/distilhubert"
model_id = "facebook/wav2vec2-large-robust"
model_id ='openai/whisper-base'


feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=False
)

### Sampling rate

In [12]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [13]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [14]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/rock/rock.00012.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/rock/rock.00012.wav',
  'array': array([-0.00319324,  0.0242455 ,  0.06524472, ..., -0.00888791,
         -0.02288128,  0.        ]),
  'sampling_rate': 16000},
 'genre': 9}

### Mean & Variance

In [15]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: -4.55e-05, Variance: 0.00705


#### After the feature extractor

In [16]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_features']):.3}, Variance: {np.var(inputs['input_features']):.3}"
)

inputs keys: ['input_features']
Mean: 0.239, Variance: 0.112


# Preprocessing

In [17]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=False,
    )
    return inputs

In [18]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=32,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/799 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_features'],
        num_rows: 799
    })
    test: Dataset({
        features: ['genre', 'input_features'],
        num_rows: 200
    })
})

### Change the name 'genre' to 'label' of dataset

In [19]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

### Id to label & Label to id

In [20]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label

{'0': 'blues',
 '1': 'classical',
 '2': 'country',
 '3': 'disco',
 '4': 'hiphop',
 '5': 'jazz',
 '6': 'metal',
 '7': 'pop',
 '8': 'reggae',
 '9': 'rock'}

# Load the model

In [21]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-base and are newly initialized: ['model.projector.bias', 'model.classifier.weight', 'model.projector.weight', 'model.classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hugging Face login

In [22]:
from huggingface_hub import notebook_login
notebook_login()

# Training Arguments

In [28]:
from transformers import TrainingArguments

# Define other required variables
model_name = model_id.split("/")[-1]
batch_size = 4
gradient_accumulation_steps = 1
num_train_epochs = 30

# Define TrainingArguments
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-05,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    #seed=42,  # Added seed
    save_total_limit=1,  # Save only the best model
    lr_scheduler_type="linear",  # Linear learning rate scheduler
)

## Evaluations

In [29]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

## *Optimizer*

In [30]:
import torch
from transformers import get_linear_schedule_with_warmup

# Create PyTorch AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.999), eps=1e-8)

# Create linear learning rate scheduler with warmup
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=int(num_train_epochs * len(gtzan_encoded["train"]) / batch_size * 0.1),
                                            num_training_steps=num_train_epochs * len(gtzan_encoded["train"]) / batch_size)

# Trainer

In [31]:
from transformers import Trainer

# Create Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),  # Pass optimizer and scheduler
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,2.357700,1.955146,0.350000
2,2.049200,2.033277,0.270000
3,2.033100,1.919622,0.300000
4,1.373200,1.670492,0.340000
5,1.702100,1.700614,0.335000
6,1.907000,1.748916,0.360000
7,1.611000,1.534685,0.450000
8,1.198900,1.483530,0.465000
9,2.004900,1.368104,0.525000
10,0.956200,1.473201,0.490000


TrainOutput(global_step=6000, training_loss=0.9086778098154561, metrics={'train_runtime': 5165.4597, 'train_samples_per_second': 4.64, 'train_steps_per_second': 1.162, 'total_flos': 6.88834174032e+17, 'train_loss': 0.9086778098154561, 'epoch': 30.0})

# Save to hub

In [32]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)

'https://huggingface.co/PawanKrGunjan/whisper-base-finetuned-gtzan/tree/main/'